In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

/Users/jongwoom1pro/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_data = pd.read_csv("../data/train.csv")
test_data = pd.read_csv("../data/test.csv")

In [3]:
display(train_data.shape)
display(test_data.shape)

(11504798, 12)

(7669866, 11)

# Data Preprocessing

In [4]:
# 필요 없는 변수 제거
train_data.drop(['id'], axis=1, inplace=True)
test_data.drop(['id'], axis=1, inplace=True)

In [5]:
# 범주형 변수 object 로 변환
# 범주형 데이터 int -> object로 변환
train_data['Driving_License'] = train_data['Driving_License'].astype('object')
train_data['Region_Code'] = train_data['Region_Code'].astype('object')
train_data['Previously_Insured'] = train_data['Previously_Insured'].astype('object')
train_data['Policy_Sales_Channel'] = train_data['Policy_Sales_Channel'].astype('object')
#train_data['Response'] = train_data['Response'].astype('object')

test_data['Driving_License'] = test_data['Driving_License'].astype('object')
test_data['Region_Code'] = test_data['Region_Code'].astype('object')
test_data['Previously_Insured'] = test_data['Previously_Insured'].astype('object')
test_data['Policy_Sales_Channel'] = test_data['Policy_Sales_Channel'].astype('object')

In [6]:
# 데이터 분할
x_train = train_data.drop(['Response'], axis=1)
y_train = train_data['Response']
x_test = test_data

# 범주형 데이터 더미처리
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

/Users/jongwoom1pro/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/Users/jongwoom1pro/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [7]:
# 학습 데이터와 테스트 데이터에서 동일한 컬럼을 유지
x_train, x_test = x_train.align(x_test, join='outer', axis=1, fill_value=0)

In [8]:
# 데이터 샘플링 (예: 10% 데이터만 사용)
sample_fraction = 0.01  # 샘플 비율 (1%)
x_sample, y_sample = x_train[:int(sample_fraction * len(x_train))], y_train[:int(sample_fraction * len(y_train))]


In [9]:
# 모델 불러오기
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


# 모델 정의
model = RandomForestClassifier()

# Val DataSet으로 성능 확인
x_tr, x_val, y_tr, y_val = train_test_split(x_sample, y_sample, test_size=0.3, random_state=42)

# 모델 학습
model.fit(x_tr, y_tr)


RandomForestClassifier()

In [16]:
# 예측
y_pred = model.predict(x_val)

# 성능 확인
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
#display(accuracy_score(y_val, y_pred))
#display(f1_score(y_val, y_pred))
#display(recall_score(y_val, y_pred))
#display(precision_score(y_val, y_pred))
display(roc_auc_score(y_val, y_pred))

0.5594732143354669

In [9]:
# pycaret 라이브러리 사용
from pycaret.classification import *


samData = pd.concat([x_sample, y_sample], axis=1)

# 실험 환경 구성
clf = setup(data = samData, target = 'Response', session_id=123, log_experiment=True, experiment_name='insurance')

# 모델 학습
best_model = compare_models()

# 모델 예측
pred_holdout = predict_model(best_model)

# 성능 확인
evaluate_model(best_model)

ImportError: MlflowLogger requires mlflow. Install using `pip install mlflow`